https://github.com/mdda/deep-learning-workshop/blob/master/notebooks/5-RNN/3-Text-Corpus-and-Embeddings.ipynb

https://www.youtube.com/watch?v=dC_GInPpvU8

In [17]:
import pandas as pd
import nltk
from nltk.tokenize import TreebankWordTokenizer

In [142]:
df = pd.read_csv('data/tesla_label_twoprice_andtopic.csv')
df.head()

,created_at,text,label_topicModels,Price_label,"Price_label(0,1)"
0,2011-01-18,watch slammed suv blowing red light mph miami ...,1,-1,0
1,2011-01-19,tesla xyecla,1,-1,0
2,2011-01-21,yep tesla going take world structurally unprofi,1,2,1
3,2011-02-01,introducing dog mode set cabin temperature kee...,0,1,1
4,2011-02-11,wheeled electricvehicle company could next tes...,1,-1,0


In [144]:
df = df.dropna()

In [145]:
len(df)

562

In [138]:
#nltk.download('punkt')

In [15]:
#sentence_splitter = nltk.data.load('tokenizers/punkt/english.pickle')
#sentence_splitter.tokenize("difference going stellar marketp")

['difference going stellar marketp']

In [18]:
tokenizer = TreebankWordTokenizer()
tokenizer.tokenize("This is Mr. Smith's tokenized test.")

['This', 'is', 'Mr.', 'Smith', "'s", 'tokenized', 'test', '.']

In [22]:
# datastories.twitter.200d.txt
SENTENCE_LENGTH_MAX = 140
EMBEDDING_DIM=50

In [92]:
#' | '.join(next(corpus_sentence_tokens_gen))

In [146]:
def corpus_sentence_tokens_df(df):
    for text in df:
        tree_banked = tokenizer.tokenize(text)
        if len(tree_banked) < SENTENCE_LENGTH_MAX:
                        yield tree_banked
corpus_sentence_tokens_gen = corpus_sentence_tokens_df(df['text'])

In [147]:
' | '.join(next(corpus_sentence_tokens_gen))

'watch | slammed | suv | blowing | red | light | mph | miami | beach | somehow | driver'

In [148]:
for i in corpus_sentence_tokens_gen:
    pass
    #print(i)

In [150]:
from nltk.tag.perceptron import PerceptronTagger
pos_tagger = PerceptronTagger(load=True)
#' | '.join(list(pos_tagger.classes))

In [151]:
df['pos_tagger'] = df['text'].apply(pos_tagger.tag)

In [ ]:
s = "Let 's see what part of speech analysis on Jeff 's sample text looks like .".split(' ')
#s = next(corpus_sentence_tokens_gen)
pos_tagger.tag(s)

# GloVe Word Embeddings

For speed, this looks at the first 100,000 tokens in the corpus - and should create the co-occurences in 30 seconds or so.

In [47]:
import glove
import time

In [48]:
#! pip install glove_python

## Create the Co-occurrence Matrix

In [135]:
glove_corpus = glove.Corpus()
corpus_sentences = [ w for w in corpus_sentence_tokens_gen]
       

# Fit the co-occurrence matrix using a sliding window of 10 words.
t0 = time.time()
glove_corpus.fit(corpus_sentences, window=10)

print("Dictionary length=%d" % (len(glove_corpus.dictionary),))
print("Co-occurrence calculated in %5.1fsec" % (time.time()-t0, ))

ValueError: not enough values to unpack (expected 2, got 1)

In [152]:
glove_corpus = glove.Corpus()
corpus_sentences = [ w for w in df['pos_tagger']]
       

# Fit the co-occurrence matrix using a sliding window of 10 words.
t0 = time.time()
glove_corpus.fit(corpus_sentences, window=10)

print("Dictionary length=%d" % (len(glove_corpus.dictionary),))
print("Co-occurrence calculated in %5.1fsec" % (time.time()-t0, ))

Dictionary length=166
Co-occurrence calculated in   0.0sec


In [154]:
# Return the index of the word in the dictionary
#glove_corpus.dictionary['tesla']

## Create the Word Embedding

In [155]:
word_embedding = glove.Glove(no_components=EMBEDDING_DIM, learning_rate=0.05)

t0 = time.time()
glove_epochs, glove_threads = 20, 4 

word_embedding.fit(glove_corpus.matrix, epochs=glove_epochs, no_threads=glove_threads, verbose=True)

print("%d-d word-embedding created in %5.1fsec = %5.1fsec per epoch" % (
        EMBEDDING_DIM, (time.time()-t0), (time.time()-t0)/glove_epochs*glove_threads, ))

# Add the word -> id dictionary to the model to allow similarity queries.
word_embedding.add_dictionary(glove_corpus.dictionary)

Performing 20 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
50-d word-embedding created in   0.1sec =   0.0sec per epoch


In [ ]:
#word_embedding.save("./data/RNN/glove.embedding.50.pkl")
#word_embedding.load("./data/RNN/glove.embedding.50.pkl")


# Test Word Embedding¶

In [ ]:
# word-similarity test
word_embedding.most_similar('country')

In [ ]:
# word-analogy test
def get_embedding_vec(word):
    idx = word_embedding.dictionary.get(word.lower(), -1)
    if idx<0:
        #print("Missing word : '%s'" % (word,))
        return np.zeros(  (EMBEDDING_DIM, ), dtype='float32')  # UNK
    return word_embedding.word_vectors[idx]

def get_closest_word(vec, number=5):
    dst = (np.dot(word_embedding.word_vectors, vec)
                   / np.linalg.norm(word_embedding.word_vectors, axis=1)
                   / np.linalg.norm(vec))
    word_ids = np.argsort(-dst)
    return [(word_embedding.inverse_dictionary[x], dst[x]) for x in word_ids[:number]
            if x in word_embedding.inverse_dictionary]

In [ ]:
analogy_vec = get_embedding_vec('woman') + get_embedding_vec('king') - get_embedding_vec('man')
get_closest_word(analogy_vec)

In [ ]:
def test_analogy(s='one two three four'):
    (a,b,c,d) = s.split(' ')
    analogy_vec = get_embedding_vec(b) - get_embedding_vec(a) + get_embedding_vec(c)
    words = [ w for (w,p) in get_closest_word(analogy_vec) if w not in (a,b,c)]
    print("'%s' is to '%s' as '%s' is to {%s}" % (a,b,c,', '.join(words)))

In [ ]:
test_analogy('man woman king queen')
test_analogy('paris france rome italy')
test_analogy('kitten cat puppy dog')
test_analogy('understand understood run ran')

## Problem : Embedding is Poor

## Solution : Load a pre-trained word embedding


Since the embedding we learnt above is poor, let's load a pre-trained word embedding, from a much larger corpus, trained for a much longer period. Source of this word embedding (created from a 6 billion tokens corpus, with results as 50d vectors): http://nlp.stanford.edu/projects/glove/

NB: If you don't have the required data, and the RedCatLabs server doesn't give you the download, the loader below downloads a 823Mb file via a fairly slow connection to a server at Stanford (this can take HOURS).

In [156]:
import os, requests, shutil

glove_dir = './data/RNN/'
glove_100k_50d = 'glove.first-100k.6B.50d.txt'
glove_100k_50d_path = os.path.join(glove_dir, glove_100k_50d)

# These are temporary files if we need to download it from the original source (slow)
data_cache = './data/cache'
glove_full_tar = 'glove.6B.zip'
glove_full_50d = 'glove.6B.50d.txt'

#force_download_from_original=False
download_url= 'http://redcatlabs.com/downloads/deep-learning-workshop/notebooks/data/RNN/'+glove_100k_50d
original_url = 'http://nlp.stanford.edu/data/'+glove_full_tar

if not os.path.isfile( glove_100k_50d_path ):
    if not os.path.exists(glove_dir):
        os.makedirs(glove_dir)
    
    # First, try to download a pre-prepared file directly...
    response = requests.get(download_url, stream=True)
    if response.status_code == requests.codes.ok:
        print("Downloading 42Mb pre-prepared GloVE file from RedCatLabs")
        with open(glove_100k_50d_path, 'wb') as out_file:
            shutil.copyfileobj(response.raw, out_file)
    else:
        # But, for some reason, RedCatLabs didn't give us the file directly
        if not os.path.exists(data_cache):
            os.makedirs(data_cache)
        
        if not os.path.isfile( os.path.join(data_cache, glove_full_50d) ):
            zipfilepath = os.path.join(data_cache, glove_full_tar)
            if not os.path.isfile( zipfilepath ):
                print("Downloading 860Mb GloVE file from Stanford")
                response = requests.get(download_url, stream=True)
                with open(zipfilepath, 'wb') as out_file:
                    shutil.copyfileobj(response.raw, out_file)
            if os.path.isfile(zipfilepath):
                print("Unpacking 50d GloVE file from zip")
                import zipfile
                zipfile.ZipFile(zipfilepath, 'r').extract(glove_full_50d, data_cache)

        with open(os.path.join(data_cache, glove_full_50d), 'rt') as in_file:
            with open(glove_100k_50d_path, 'wt') as out_file:
                print("Reducing 50d GloVE file to first 100k words")
                for i, l in enumerate(in_file.readlines()):
                    if i>=100000: break
                    out_file.write(l)
    
        # Get rid of tarfile source (the required text file itself will remain)
        #os.unlink(zipfilepath)
        #os.unlink(os.path.join(data_cache, glove_full_50d))

print("GloVE available locally")

GloVE available locally


In [110]:
word_embedding = glove.Glove.load_stanford( glove_100k_50d_path )
word_embedding.word_vectors.shape

(100000, 50)

In [112]:
word_embedding.most_similar('tesla')

[('volt', 0.6956429105311589),
 ('westinghouse', 0.678256119091891),
 ('ev1', 0.6599545102145767),
 ('sandisk', 0.6567678114984614)]

In [113]:
test_analogy('man woman king queen')
test_analogy('paris france rome italy')
test_analogy('kitten cat puppy dog')
test_analogy('understand understood run ran')

NameError: name 'test_analogy' is not defined

# Visualize Embedding in TensorBoard

In [157]:
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector

#N = 10000 # Number of items (vocab size).
#D = 200 # Dimensionality of the embedding.
#embedding_var = tf.Variable(tf.random_normal([N,D]), name='word_embedding')

embedding_var = tf.Variable(word_embedding.word_vectors, dtype='float32', 
                            name='word_embedding')

# Format: tensorflow/contrib/tensorboard/plugins/projector/projector_config.proto
projector_config = projector.ProjectorConfig()

# You can add multiple embeddings. Here we add only one.
embedding = projector_config.embeddings.add()
embedding.tensor_name = embedding_var.name

# Link this tensor to its metadata file (e.g. labels).
LOG_DIR='../../tensorflow.logdir/'
os.makedirs(LOG_DIR, exist_ok=True)    

metadata_file = 'glove_full_50d.words.tsv'
vocab_list = [ word_embedding.inverse_dictionary[i] 
               for i in range(len( word_embedding.inverse_dictionary )) ]
with open(os.path.join(LOG_DIR, metadata_file), 'wt') as metadata:
    metadata.writelines("%s\n" % w for w in vocab_list)
    
embedding.metadata_path = os.path.join(os.getcwd(), LOG_DIR, metadata_file)

# Use the same LOG_DIR where you stored your checkpoint.
summary_writer = tf.summary.FileWriter(LOG_DIR)

# The next line writes a projector_config.pbtxt in the LOG_DIR. TensorBoard will
# read this file during startup.
projector.visualize_embeddings(summary_writer, projector_config)

saver = tf.train.Saver([embedding_var])

with tf.Session() as sess:
    # Initialize the model
    sess.run(tf.global_variables_initializer())
    
    saver.save(sess, os.path.join(LOG_DIR, metadata_file+'.ckpt'))
print("Look at the embedding in TensorBoard : http://localhost:8081/")

TypeError: not all arguments converted during string formatting

# Run TensorBoard via Colab

In [128]:
# Start the tensorboard server on this (colab) machine
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 8081 &'
    .format(LOG_DIR)
)

In [129]:
#! npm install localtunnel

In [130]:
! ls -l node_modules/localtunnel/bin/client

ls: node_modules/localtunnel/bin/client: No such file or directory


In [131]:

# Tunnel port 8081 (TensorBoard assumed running)
get_ipython().system_raw('node_modules/localtunnel/bin/client --port 8081 >> tunnel_url.txt 2>&1 &')


In [132]:
! ps fax | grep node | grep 8081

ps: illegal option -- f
usage: ps [-AaCcEefhjlMmrSTvwXx] [-O fmt | -o fmt] [-G gid[,gid...]]
          [-u]
          [-p pid[,pid...]] [-t tty[,tty...]] [-U user[,user...]]
       ps [-L]


In [133]:
! cat tunnel_url.txt

/bin/bash: node_modules/localtunnel/bin/client: No such file or directory
/bin/bash: node_modules/localtunnel/bin/client: No such file or directory
/bin/bash: node_modules/localtunnel/bin/client: No such file or directory


In [ ]:

import numpy as np
import pandas as pd
from tqdm import tqdm
from  ELmo import ELMo_google_model
import re
import logging
logging.getLogger('tensorflow').disabled = True #OPTIONAL - to disable outputs from Tensorflow

number = 1000

df_data = pd.read_csv('/Users/apple/Documents/Programming/python/Project/data/tweets/Secound_CLean_training.1600000.processed.noemoticon.csv')
df_data['text'] = df_data['text'].apply(lambda x: re.sub('-PRON- ','',str(x)))
df_data.head()

df = df_data[:number]
""" ELMo """
if __name__ == "__main__":
    #for i in tqdm(range(int(len(df)/100))):
    Elmo = ELMo_google_model(df,name='text',dimension=50)
    x= Elmo.ELMo_Embadding()
    #x = Elmo.reducation_dimensional(dimension=50)
    Elmo.Save_ELMo(namefile='data/Elmo_tweets_badgood_label.csv')



In [ ]:
x

In [10]:
from sklearn.decomposition import PCA
pca = PCA(n_components=50) #reduce down to 50 dim
y = pca.fit_transform(x[0:2])

In [11]:
y

array([[-3.4377410e+00,  1.7933664e-06],
       [ 3.4377449e+00,  1.7933646e-06]], dtype=float32)

In [1]:
from farhad.Farhadcolor import Text_colors,background_colors 

In [2]:
print(background_colors.RED+' a car')
print('is in road')
print(bcolors.ENDC)
print('the car drived by  a man')

 a car
is in road


NameError: name 'bcolors' is not defined

In [4]:
from gensim.models import Word2Vec

In [ ]:
# https://tfhub.dev/google/nnlm-en-dim50-with-normalization/1

In [5]:
wv =  Word2Vec.load_word2vec_format('https://tfhub.dev/google/nnlm-en-dim50-with-normalization/1')

DeprecationWarning: Deprecated. Use gensim.models.KeyedVectors.load_word2vec_format instead.

# ----------/A\

In [1]:
import optparse
import zipfile
from threading import  Thread
from farhad.Farhadcolor import tcolors,bcolors

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [ ]:
def extract_zip(Zfile, password):
    try:
        Zfile.extractall(pwd=password)
        print(tcolors.RED+"[+] Password Found: "+password+tcolors.ENDC+'\n')
    except:
        pass
    
def Main():
    parser = optparse.OptionParser('usage sprong'+'-f <zipfile> -d <dictionary>')
    parser.add_option('-f',dest='zname',type='string',help='specify zip file')
    parser.add_option('-d',dest='dname',type='string',help='specify dictionary file')
    
    (option,arg)=parser.prase_args()
    if option.zname == None or option.dname == None:
        print(parser.usage)
        exit(0)
    else:
        zname = option.zname
        dname = option.dname
        
    zfile = zipfile.ZipFile(zname) 
    passFile = open(dname)
    for i  in passFile.readlines():
        password = line.split('\n')
        t = Thread(target=extratc_zip, args=(zFile,passwrod))
        t.start()

In [ ]:
if __name__ == '__main__':
    Main()

In [6]:

from farhad.TwCleaner import Tweets_preprocesiing
import pandas as pd
from IPython.display import HTML
from IPython.display import display
from farhad.Farhadcolor import tcolors,bcolors




if __name__ == "__main__":
    print(tcolors.BLUE)
    display(HTML('<h2 style="color:red"><b>input your file:</h2></b>'))
    path_file = '/Users/apple/Documents/Programming/python/Project/data/tweets/training.1600000.processed.noemoticon.csv'
    df = pd.read_csv(path_file,na_values=['nan','?'])
    df = df.dropna()
    print(bcolors.BLUE+tcolors.WHITE,'coloumns:',df.columns,tcolors.ENDC)
    print(bcolors.BLUE+tcolors.WHITE,'info: \n',df.info(),tcolors.ENDC)
    print(bcolors.BLUE+tcolors.WHITE,'lenght: ',len(df),tcolors.ENDC)
    print(tcolors.BLUE)
    lenght = input('how lenght should be?')


    display(HTML("<h2 style='color:red'><b>one of columns:</h2></b> "))
    target = str(input('one of columns:'))
    display(HTML('<h2 style="color:red"><b>save file path:</h2></b>'))
    save_file = str(input('save file path:'))
    if 'created_at' is not df.columns:
        display(HTML("<h2 style='color:red'><b>write name of date column:</h2></b>"))
        datee = str(input('write name of date column:'))
    else:
        datee = 'created_at'

    CT = Tweets_preprocesiing()
    data1  = CT.Cleaner(df,target=target,date=datee)
    data   = CT.Remove_stop_words(data1)
    new_df = CT.save_clean_tweets(df,data,created_at=datee,name=save_file)
    
    print('\033[0m')

UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 80-81: invalid continuation byte